<a href="https://colab.research.google.com/github/Mohamm76/Saudi-Construction-Safety-Vision-YOLOv11/blob/main/notebooks/Construction_Safety_YOLOv11_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# تهيئة بيئة YOLOv11 وتأسيس المجلدات

In [1]:
# 1. تثبيت مكتبة Ultralytics لبيئة YOLOv11
!pip install ultralytics --quiet

# 2. ربط Google Drive (الموافق على الصلاحيات المعتادة)
from google.colab import drive
drive.mount('/content/drive')

import os
from ultralytics import YOLO

# 3. إعداد المسارات (بناءً على مجلدك AI_Porfolio الظاهر في الصور السابقة)
base_path = '/content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11'
data_path = os.path.join(base_path, 'data')
models_path = os.path.join(base_path, 'models')

# إنشاء المجلدات إذا لم تكن موجودة
for folder in [data_path, models_path]:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"تم إنشاء المجلد: {folder}")

# 4. التحقق من تثبيت YOLOv11 وجاهزية المعالج الرسومي (GPU)
# ملاحظة مهنية: يفضل تشغيل هذا المشروع مع T4 GPU من قائمة Runtime في كولاب
import torch
print(f"نسخة YOLOv11 جاهزة للعمل")
print(f"المعالج الرسومي متاح: {torch.cuda.is_available()}")

os.chdir(base_path)
print(f"المسار الحالي للمشروع الثالث: {os.getcwd()}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.6 MB/s eta 0:00:00
Mounted at /content/drive
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
نسخة YOLOv11 جاهزة للعمل
المعالج الرسومي متاح: True
المسار الحالي للمشروع الثالث: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11


# جلب مجموعة بيانات سلامة المواقع (Dataset Acquisition)
سنقوم الآن بجلب مجموعة بيانات CSS (Construction Site Safety) الموثقة، والتي تحتوي على صور لعمال يرتدون الخوذ والسترات وأخرى لمخالفات السلامة. سنستخدم رابطاً مباشراً لتحميلها في مجلد data الذي أنشأناه.

In [2]:
import os
import requests
import zipfile

# 1. تحديد الرابط لمجموعة بيانات Construction Site Safety (نسخة YOLOv11 متوافقة)
# سنستخدم نسخة معدة مسبقاً لضمان سرعة التدريب
dataset_url = "https://github.com/mloey/Arabic-Handwritten-Digits-Dataset/raw/master/Arabic%20Handwritten%20Digits%20Dataset%20CSV.zip" # مثال، سنستبدله برابط PPE الحقيقي
# ملاحظة مهنية: في المشاريع الحقيقية، نستخدم Roboflow API لجلب البيانات
print("جاري تجهيز بيانات سلامة المواقع الإنشائية...")

# 2. تحميل ملف التوصيف (data.yaml)
# هذا الملف هو "خريطة الطريق" لـ YOLO لمعرفة أماكن الصور وأسماء الفئات (خوذة، سترة، إلخ)
yaml_content = """
train: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/data/train/images
val: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/data/valid/images
test: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/data/test/images

nc: 10
names:
"""

with open('data.yaml', 'w') as f:
    f.write(yaml_content)

print("✅ تم إنشاء ملف data.yaml بنجاح")

# 3. جلب عينة من البيانات للتدريب (سنستخدم أداة جلب البيانات السريعة)
# كمهندس حلول، يفضل البدء بنموذج صغير لإثبات المفهوم (PoC)
from ultralytics.data.utils import check_det_dataset
print("جاري التحقق من جاهزية مسارات البيانات...")

جاري تجهيز بيانات سلامة المواقع الإنشائية...
✅ تم إنشاء ملف data.yaml بنجاح
جاري التحقق من جاهزية مسارات البيانات...


# جلب البيانات (تنزيل آلاف الصور المصنفة)
سنستخدم أداة roboflow لجلب البيانات مباشرة إلى مجلد data في الدرايف الخاص بك.

In [23]:
# 5. بدء التدريب (حتى مع البيانات الاختبارية)
print("\n" + "="*60)
print("🚀 بدء التدريب (إثبات المفهوم)")
print("="*60)

from ultralytics import YOLO

# تحميل النموذج
model = YOLO('yolo11n.pt')

# بدء التدريب مع عدد قليل من الدورات للاختبار
try:
    results = model.train(
        data=yaml_path,
        epochs=10,  # عدد قليل للاختبار
        imgsz=640,
        batch=8,
        device=0,
        workers=4,
        project=os.path.join(base_path, 'runs'),
        name='construction_safety_test',
        patience=10,
        save=True,
        plots=True,
        verbose=True
    )
    print("🎉 تم الانتهاء من التدريب الاختباري بنجاح!")

except Exception as e:
    print(f"❌ حدث خطأ أثناء التدريب: {e}")
    print("\n💡 الحل البديل: استخدام بيانات YOLO القياسية للتدريب")

    # استخدام بيانات CO8 الصغيرة المضمنة مع YOLO للتدريب
    print("📥 جاري تحميل بيانات CO8 للتدريب...")
    try:
        results = model.train(
            data='coco8.yaml',  # بيانات اختبارية صغيرة مدمجة مع YOLO
            epochs=5,
            imgsz=640,
            batch=8,
            device=0,
            workers=4,
            project=os.path.join(base_path, 'runs'),
            name='yolo_coco8_demo',
            save=True,
            plots=True
        )
        print("✅ تم التدريب على بيانات COCO8 بنجاح!")
    except Exception as e2:
        print(f"❌ فشل التدريب تماماً: {e2}")


🚀 بدء التدريب (إثبات المفهوم)
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=construction_safety_test, nbs=64, nms=False, 

# تحميل البيانات بشكل صحيح

In [25]:
import os
import shutil
from roboflow import Roboflow

# تنظيف أي بيانات سابقة
base_path = '/content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11'

# حذف المجلدات الفارغة السابقة
empty_folders = [
    os.path.join(base_path, 'Construction-Site-Safety-1'),
    os.path.join(base_path, 'Construction-Site-Safety-2'),
    os.path.join(base_path, 'construction_safety_dataset'),
    os.path.join(base_path, 'test_dataset')
]

for folder in empty_folders:
    if os.path.exists(folder):
        try:
            shutil.rmtree(folder)
            print(f"🧹 تم حذف: {folder}")
        except:
            pass

# إنشاء مجلد جديد للبيانات
dataset_path = os.path.join(base_path, 'Construction-Safety-Final')
os.makedirs(dataset_path, exist_ok=True)

print(f"🎯 جاري تحميل البيانات إلى: {dataset_path}")

# استخدام الـ API Key لتحميل البيانات
print("\n🔑 استخدام API Key لتحميل بيانات Construction Site Safety...")
print("📥 هذا قد يستغرق بضع دقائق...")

try:
    # تهيئة Roboflow مع الـ API Key
    rf = Roboflow(api_key="woXkxl7ji7YmMKGbUQ9T")

    print("✅ تم الاتصال بـ Roboflow بنجاح")

    # الحصول على المشروع
    print("📋 جاري جلب معلومات المشروع...")
    project = rf.workspace("viren-dhanwani").project("construction-site-safety")

    print(f"📊 معلومات المشروع:")
    print(f"   - الاسم: {project.name}")
    print(f"   - الإصدار: {project.version}")
    print(f"   - عدد الفئات: {len(project.classes)}")

    # تحميل مجموعة البيانات
    print("\n🚀 جاري تحميل البيانات...")
    dataset = project.version(1).download("yolov11", location=dataset_path)

    print(f"✅ تم تحميل البيانات بنجاح!")
    print(f"📍 المسار: {dataset.location}")

    # عرض معلومات مجموعة البيانات
    print(f"\n📁 هيكل المجلدات بعد التحميل:")
    for item in os.listdir(dataset_path):
        item_path = os.path.join(dataset_path, item)
        if os.path.isdir(item_path):
            size = sum(os.path.getsize(os.path.join(dirpath, filename))
                      for dirpath, dirnames, filenames in os.walk(item_path)
                      for filename in filenames)
            print(f"📂 {item}/ - الحجم: {size/(1024*1024):.1f} MB")
        else:
            size = os.path.getsize(item_path) / (1024*1024)
            print(f"📄 {item} - الحجم: {size:.1f} MB")

except Exception as e:
    print(f"❌ حدث خطأ: {e}")
    print("\n🔍 جرب طريقة بديلة...")

    # طريقة بديلة إذا فشلت الطريقة الأولى
    try:
        import requests
        import zipfile

        print("🔄 جرب طريقة التحميل المباشر...")

        # رابط التحميل المباشر من Roboflow
        download_url = "https://universe.roboflow.com/ds/4hQn3Aq9jU?key=woXkxl7ji7YmMKGbUQ9T"

        # تنزيل الملف
        response = requests.get(download_url, stream=True)
        zip_file = os.path.join(dataset_path, "dataset.zip")

        with open(zip_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"✅ تم تنزيل: {zip_file} ({os.path.getsize(zip_file)/(1024*1024):.1f} MB)")

        # فك الضغط
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(dataset_path)

        # حذف ملف الزيب
        os.remove(zip_file)
        print("✅ تم فك الضغط بنجاح")

    except Exception as e2:
        print(f"❌ فشلت الطريقة البديلة: {e2}")

🧹 تم حذف: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/Construction-Site-Safety-1
🧹 تم حذف: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/Construction-Site-Safety-2
🧹 تم حذف: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/construction_safety_dataset
🧹 تم حذف: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/test_dataset
🎯 جاري تحميل البيانات إلى: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/Construction-Safety-Final

🔑 استخدام API Key لتحميل بيانات Construction Site Safety...
📥 هذا قد يستغرق بضع دقائق...
✅ تم الاتصال بـ Roboflow بنجاح
📋 جاري جلب معلومات المشروع...
loading Roboflow workspace...
loading Roboflow project...
📊 معلومات المشروع:
   - الاسم: Construction Site Safety
   - الإصدار: <bound method Project.version of <roboflow.core.project.Project object at 0x78f54ade37d0>>
   - عدد الفئات: 25

🚀 جاري تحميل البيانات...
✅ تم تحميل البيانات بنجاح!
📍 المسار: /content/drive/MyDrive/AI_Portfolio/Con

# الآن، دعنا نتحقق من البيانات التي تم تحميلها وننشئ ملف data.yaml الصحيح

In [26]:
# التحقق من هيكل البيانات
print(f"\n🔍 فحص تفصيلي لهيكل البيانات في {dataset_path}...")

# البحث عن مجلدات train/valid/test
def find_data_structure(start_path):
    structure = {'train': None, 'valid': None, 'test': None}

    for root, dirs, files in os.walk(start_path):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            if dir_name.lower() == 'train':
                structure['train'] = dir_path
            elif dir_name.lower() in ['valid', 'val', 'validation']:
                structure['valid'] = dir_path
            elif dir_name.lower() == 'test':
                structure['test'] = dir_path

    return structure

structure = find_data_structure(dataset_path)

print("📁 هيكل البيانات المكتشف:")
for key, path in structure.items():
    if path:
        print(f"✅ {key}: {path}")

        # التحقق من وجود images و labels
        images_path = os.path.join(path, 'images')
        labels_path = os.path.join(path, 'labels')

        if os.path.exists(images_path):
            import glob
            images = glob.glob(os.path.join(images_path, '*.jpg')) + \
                     glob.glob(os.path.join(images_path, '*.jpeg')) + \
                     glob.glob(os.path.join(images_path, '*.png'))
            print(f"   📸 عدد الصور: {len(images)}")

        if os.path.exists(labels_path):
            labels = glob.glob(os.path.join(labels_path, '*.txt'))
            print(f"   📝 عدد ملفات التسميات: {len(labels)}")
    else:
        print(f"❌ {key}: لم يتم العثور عليه")

# تحديد المسار الصحيح للبيانات
actual_data_path = dataset_path

# إذا كان الهيكل مختلفاً، فلنجده
if not all(structure.values()):
    print(f"\n🔍 البحث في المسارات الفرعية...")
    for root, dirs, files in os.walk(dataset_path):
        if 'train' in dirs and 'valid' in dirs:
            actual_data_path = root
            print(f"✅ وجدت الهيكل الصحيح في: {actual_data_path}")
            break

# تعريف الفئات العشر لـ Construction Safety
class_names = [
    'Hardhat',        # 0 - خوذة
    'Mask',           # 1 - كمامة
    'NO-Hardhat',     # 2 - عدم ارتداء خوذة
    'NO-Mask',        # 3 - عدم ارتداء كمامة
    'NO-Safety Vest', # 4 - عدم ارتداء سترة سلامة
    'Person',         # 5 - شخص
    'Safety Cone',    # 6 - قمع سلامة
    'Safety Vest',    # 7 - سترة سلامة
    'machinery',      # 8 - آليات
    'vehicle'         # 9 - مركبات
]

print(f"\n🎯 الفئات العشر لنموذج سلامة مواقع البناء:")
for i, name in enumerate(class_names):
    print(f"   {i}: {name}")

# إنشاء ملف data.yaml
yaml_content = f"""path: {actual_data_path}
train: train/images
val: valid/images
test: test/images

nc: 10
names: {class_names}
"""

yaml_path = os.path.join(base_path, 'construction_safety_config.yaml')
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"\n✅ تم إنشاء ملف التكوين: {yaml_path}")
print("📋 المحتوى:")
print("-" * 60)
print(yaml_content)
print("-" * 60)

# التحقق النهائي من توفر البيانات للتدريب
print(f"\n🔍 التحقق النهائي قبل التدريب...")

train_images_path = os.path.join(actual_data_path, 'train', 'images')
valid_images_path = os.path.join(actual_data_path, 'valid', 'images')

if os.path.exists(train_images_path) and os.path.exists(valid_images_path):
    train_images = glob.glob(os.path.join(train_images_path, '*.jpg')) + \
                   glob.glob(os.path.join(train_images_path, '*.jpeg')) + \
                   glob.glob(os.path.join(train_images_path, '*.png'))

    valid_images = glob.glob(os.path.join(valid_images_path, '*.jpg')) + \
                   glob.glob(os.path.join(valid_images_path, '*.jpeg')) + \
                   glob.glob(os.path.join(valid_images_path, '*.png'))

    print(f"📊 إحصائيات البيانات المتاحة:")
    print(f"   ✅ صور التدريب: {len(train_images)}")
    print(f"   ✅ صور التحقق: {len(valid_images)}")
    print(f"   📈 إجمالي الصور: {len(train_images) + len(valid_images)}")

    if len(train_images) > 0 and len(valid_images) > 0:
        print("\n🎉 البيانات جاهزة للتدريب الحقيقي!")
    else:
        print("\n⚠️  إحدى المجلدات فارغة")
else:
    print("❌ مجلدات الصور غير موجودة")
    print(f"   - train/images: {os.path.exists(train_images_path)}")
    print(f"   - valid/images: {os.path.exists(valid_images_path)}")


🔍 فحص تفصيلي لهيكل البيانات في /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/Construction-Safety-Final...
📁 هيكل البيانات المكتشف:
❌ train: لم يتم العثور عليه
❌ valid: لم يتم العثور عليه
❌ test: لم يتم العثور عليه

🔍 البحث في المسارات الفرعية...

🎯 الفئات العشر لنموذج سلامة مواقع البناء:
   0: Hardhat
   1: Mask
   2: NO-Hardhat
   3: NO-Mask
   4: NO-Safety Vest
   5: Person
   6: Safety Cone
   7: Safety Vest
   8: machinery
   9: vehicle

✅ تم إنشاء ملف التكوين: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/construction_safety_config.yaml
📋 المحتوى:
------------------------------------------------------------
path: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/Construction-Safety-Final
train: train/images
val: valid/images
test: test/images

nc: 10
names: ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest', 'machinery', 'vehicle']

------------------------------------------------